In [357]:
import pandas as pd
import sys
sys.path.append('..')
from src.timeconverter import date_to_timestamp


target = pd.read_csv('../train/target/y_predict.csv')
print(len(target['wagnum']), len(target['wagnum'].unique()))

33707 33707


In [358]:
wag_param = pd.read_csv('../preprocessed/wag_param.csv', index_col = 0)
kti = pd.read_csv('../preprocessed/kti.csv', index_col = 0)

train = pd.read_csv('./target/y_train.csv')
train['month'] = train['month'].apply(date_to_timestamp)
test = pd.read_csv('../train2/target/y_test.csv')
test['month'] = test['month'].apply(date_to_timestamp)

In [360]:
features = pd.merge(wag_param, kti, on='wagnum', how='left')
features

,wagnum,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,zavod_build,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,26318,12-600-04,1,682,682,85.0,240,725241600,1651017600,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28344,12-132,1,700,700,88.0,240,1060646400,1734998400,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8099,11-286,0,670,670,138.0,270,809827200,1822348800,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33350,12-9850-02,1,750,750,90.0,248,1414368000,2432937600,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5308,11-276,0,680,680,122.0,260,811296000,1822089600,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95110,18914,11-280,0,680,680,138.0,259,1371340800,2414102400,1,...,27.6,28.1,48.2,50.4,48.3,48.8,42.9,45.0,42.2,44.5
95111,18914,11-280,0,680,680,138.0,259,1371340800,2414102400,1,...,27.5,27.5,49.9,49.4,49.5,49.5,42.6,43.8,43.6,44.3
95112,18914,11-280,0,680,680,138.0,259,1371340800,2414102400,1,...,28.1,27.8,45.9,47.7,46.5,48.5,42.9,43.1,40.6,44.0
95113,18914,11-280,0,680,680,138.0,259,1371340800,2414102400,1,...,27.6,27.5,47.2,47.4,48.2,49.0,43.4,42.9,42.5,44.2


In [339]:
features.info()

cat_features = ['model','zavod_build', 'date_iskl', 'kuzov', 'telega', 'tormoz', 'tipvozd']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95115 entries, 0 to 95114
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   wagnum               95115 non-null  int64  
 1   model                95115 non-null  object 
 2   rod_id               95115 non-null  int64  
 3   gruz                 95115 non-null  int64  
 4   cnsi_gruz_capacity   95115 non-null  int64  
 5   cnsi_volumek         95115 non-null  float64
 6   tara                 95115 non-null  int64  
 7   date_build           95115 non-null  int64  
 8   srok_sl              95115 non-null  int64  
 9   zavod_build          95115 non-null  int64  
 10  date_iskl            95115 non-null  int64  
 11  cnsi_probeg_dr       95115 non-null  int64  
 12  cnsi_probeg_kr       95115 non-null  int64  
 13  kuzov                95115 non-null  int64  
 14  telega               95115 non-null  int64  
 15  tormoz               95115 non-null 

In [340]:
nan_cols = ['axl1_l_w_flange',
       'axl1_r_w_flange', 'axl2_l_w_flange', 'axl2_r_w_flange',
       'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange',
       'axl4_r_w_flange', 'axl1_l_w_rim', 'axl1_r_w_rim', 'axl2_l_w_rim',
       'axl2_r_w_rim', 'axl3_l_w_rim', 'axl3_r_w_rim', 'axl4_l_w_rim',
       'axl4_r_w_rim']


def preprocessing(data, test=False):
    
    data = pd.merge(data, features, on='wagnum', how='left')
    data['operation_date_dttm'].fillna(data['month'], inplace=True)
    data[data['month'] >= data['operation_date_dttm']]
    means_nan_cols = data[nan_cols].mean()
    mileage_mean = round(data['mileage_all'].mean())
    data['mileage_all'].fillna(mileage_mean, inplace=True)
    data[nan_cols] = data[nan_cols].fillna(means_nan_cols)

    
    return data

In [341]:
train = preprocessing(train)

print(train.isna().sum().sum())
train

0


,wagnum,month,target_month,target_day,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.200000,59.90000,59.100000,58.8000
1,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.300000,57.10000,57.100000,54.5000
2,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.200000,58.50000,57.400000,57.0000
3,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.200000,58.80000,56.900000,56.9000
4,33361,1672531200,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.200000,56.10000,56.400000,53.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570676,21361,1669852800,0,0,12-783,1,700,700,76.0,234,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.900000,56.40000,59.900000,59.7000
570677,21361,1669852800,0,0,12-783,1,700,700,76.0,234,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.200000,56.70000,59.000000,59.3000
570678,21361,1669852800,0,0,12-783,1,700,700,76.0,234,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.700000,55.20000,59.100000,56.2000
570679,8061,1669852800,0,0,11-286,0,670,670,138.0,270,...,29.166353,29.218642,62.373331,62.386428,61.318671,61.344268,61.346119,61.35398,62.379559,62.3867


In [342]:
test = preprocessing(test)
print(test.isna().sum().sum())
test

0


,wagnum,month,target_month,target_day,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.20000,59.900000,59.100000,58.800000
1,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.30000,57.100000,57.100000,54.500000
2,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.20000,58.500000,57.400000,57.000000
3,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.20000,58.800000,56.900000,56.900000
4,33361,1675209600,0,0,12-1303-01,1,690,690,88.0,245,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.20000,56.100000,56.400000,53.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94591,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.300000,28.700000,68.800000,69.000000,66.200000,64.300000,58.80000,56.700000,59.900000,59.400000
94592,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.90000,56.400000,59.900000,59.700000
94593,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.20000,56.700000,59.000000,59.300000
94594,21361,1675209600,0,0,12-783,1,700,700,76.0,234,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.70000,55.200000,59.100000,56.200000


In [343]:
def split(data):
    y1 = data['target_month']
    y2 = data['target_day']

    X = data.drop(['target_month', 'target_day'], axis=1)
    return X, y1, y2

In [344]:
X_train, y1_train, y2_train = split(train)
X_train

,wagnum,month,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.200000,59.90000,59.100000,58.8000
1,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.300000,57.10000,57.100000,54.5000
2,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.200000,58.50000,57.400000,57.0000
3,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.200000,58.80000,56.900000,56.9000
4,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.200000,56.10000,56.400000,53.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570676,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.900000,56.40000,59.900000,59.7000
570677,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.200000,56.70000,59.000000,59.3000
570678,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.700000,55.20000,59.100000,56.2000
570679,8061,1669852800,11-286,0,670,670,138.0,270,757296000,1760054400,...,29.166353,29.218642,62.373331,62.386428,61.318671,61.344268,61.346119,61.35398,62.379559,62.3867


In [345]:
X_test, y1_test, y2_test = split(train)
X_test

,wagnum,month,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.200000,59.90000,59.100000,58.8000
1,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.300000,57.10000,57.100000,54.5000
2,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.200000,58.50000,57.400000,57.0000
3,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.200000,58.80000,56.900000,56.9000
4,33361,1672531200,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.200000,56.10000,56.400000,53.8000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570676,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.900000,56.40000,59.900000,59.7000
570677,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.200000,56.70000,59.000000,59.3000
570678,21361,1669852800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.700000,55.20000,59.100000,56.2000
570679,8061,1669852800,11-286,0,670,670,138.0,270,757296000,1760054400,...,29.166353,29.218642,62.373331,62.386428,61.318671,61.344268,61.346119,61.35398,62.379559,62.3867


In [346]:
# from catboost import CatBoostClassifier
# from sklearn.model_selection import StratifiedKFold

# # Assuming 'X' is your feature matrix and 'y' is your target variable

# # Define the number of folds for cross-validation
# num_folds = 5  # Change this to the desired number of folds

# # Initialize Stratified K-Fold cross-validation
# kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# # Initialize CatBoost Classifier
# model = CatBoostClassifier(iterations=500, depth=16, loss_function='Logloss', random_seed=42, cat_features=cat_features, eval_metric='F1')

# # Perform cross-validation
# for train_index, val_index in kf.split(X, y):
#     X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#     y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#     # Train the model and validate on the current fold
#     model.fit(X_train, y_train, eval_set=(X_val, y_val))


In [347]:
from catboost import CatBoostClassifier

# Initializing CatBoost Classifier
model1 = CatBoostClassifier(iterations=500, loss_function='Logloss', random_seed=42, cat_features=cat_features, eval_metric='F1')
model2 = CatBoostClassifier(iterations=500, loss_function='Logloss', random_seed=42, cat_features=cat_features, eval_metric='F1')

# Training the model
model1.fit(X_train, y1_train)
model2.fit(X_train, y2_train)
# # Model evaluation (if needed)
# metrics = model.get_evals_result()

# Feature importance
feature_importance1 = model1.get_feature_importance()
feature_importance2 = model1.get_feature_importance()

print(feature_importance1)
print(feature_importance2)

Learning rate set to 0.292387
0:	learn: 0.0000000	total: 239ms	remaining: 1m 59s
1:	learn: 0.0000000	total: 438ms	remaining: 1m 48s
2:	learn: 0.0000000	total: 642ms	remaining: 1m 46s
3:	learn: 0.0103630	total: 874ms	remaining: 1m 48s
4:	learn: 0.0064788	total: 1.08s	remaining: 1m 47s
5:	learn: 0.0099586	total: 1.26s	remaining: 1m 43s
6:	learn: 0.0113564	total: 1.46s	remaining: 1m 42s
7:	learn: 0.0200642	total: 1.64s	remaining: 1m 40s
8:	learn: 0.0200682	total: 1.84s	remaining: 1m 40s
9:	learn: 0.0229532	total: 2.01s	remaining: 1m 38s
10:	learn: 0.0376310	total: 2.2s	remaining: 1m 38s
11:	learn: 0.0384081	total: 2.42s	remaining: 1m 38s
12:	learn: 0.0387868	total: 2.63s	remaining: 1m 38s
13:	learn: 0.0397648	total: 2.82s	remaining: 1m 37s
14:	learn: 0.0315870	total: 2.99s	remaining: 1m 36s
15:	learn: 0.0315817	total: 3.21s	remaining: 1m 37s
16:	learn: 0.0407194	total: 3.46s	remaining: 1m 38s
17:	learn: 0.0433377	total: 3.64s	remaining: 1m 37s
18:	learn: 0.0435299	total: 3.85s	remaining: 

In [348]:
y1_predicted = model1.predict(X_test)
y2_predicted = model2.predict(X_test)

In [349]:
from sklearn.metrics import f1_score
score_month = f1_score(y1_predicted, y1_test)
score_day = f1_score(y2_predicted, y2_test)
score = 0.5 * score_month + 0.5 * score_day
score

0.554705339232164

Submission

In [350]:
target = preprocessing(target)
target

,wagnum,month,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,30.300000,30.000000,70.100000,67.400000,69.800000,67.600000,61.20000,59.900000,59.100000,58.800000
1,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.500000,29.800000,61.300000,60.600000,65.700000,64.100000,57.30000,57.100000,57.100000,54.500000
2,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.100000,30.800000,61.600000,61.200000,66.000000,65.100000,60.20000,58.500000,57.400000,57.000000
3,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,31.100000,62.000000,61.300000,65.300000,64.800000,57.20000,58.800000,56.900000,56.900000
4,33361,1677628800,12-1303-01,1,690,690,88.0,245,1297123200,1993248000,...,28.200000,30.400000,60.400000,60.000000,65.100000,62.700000,58.20000,56.100000,56.400000,53.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94590,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.300000,28.700000,68.800000,69.000000,66.200000,64.300000,58.80000,56.700000,59.900000,59.400000
94591,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,28.100000,68.500000,66.500000,64.400000,65.400000,57.90000,56.400000,59.900000,59.700000
94592,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.200000,28.300000,67.300000,68.300000,64.600000,64.400000,57.20000,56.700000,59.000000,59.300000
94593,21361,1677628800,12-783,1,700,700,76.0,234,1111104000,1835740800,...,29.400000,29.200000,67.800000,68.400000,65.300000,63.900000,57.70000,55.200000,59.100000,56.200000


In [352]:
y1_target = model1.predict(target)
y2_target = model2.predict(target)

In [353]:
submission = pd.read_csv('../train/target/y_predict.csv')
submission

,wagnum,month
0,33361,2023-03-01
1,33364,2023-03-01
2,33366,2023-03-01
3,33358,2023-03-01
4,33349,2023-03-01
...,...,...
33702,17621,2023-03-01
33703,25045,2023-03-01
33704,27156,2023-03-01
33705,21361,2023-03-01
